In [ ]:
%pip install flask gevent

In [1]:
!pwd

/mnt/o/study/ml_6/8.5/custom_server


In [5]:
!ls ../local/

archive		      requirements.txt
build_client.ipynb    server.log
install_server.sh     server_config.json
my_model.keras	      start_server.sh
prepare_env.sh	      tensorflow-model-server-universal_2.8.0_all.deb
prepare_model.ipynb   test.png
prepare_server.ipynb  test_images


In [6]:
!mkdir -p ./server_models/my_model/1/
!cp ../local/my_model.keras ./server_models/my_model/1/my_model.keras

In [2]:
from flask import Flask, request
from markupsafe import escape
from gevent.pywsgi import WSGIServer
import json
from pathlib import Path
import os
import re
import tensorflow as tf
import numpy as np

2024-05-27 13:25:19.596397: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-27 13:25:19.600358: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-27 13:25:20.783216: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [1]:
%%writefile server.py
from flask import Flask, request
from markupsafe import escape
from gevent.pywsgi import WSGIServer
import json
from pathlib import Path
import os
import re
import tensorflow as tf
import numpy as np


print('Current server path:', Path().resolve())
models_folder = Path("./server_models")

app = Flask(__name__)

def load_model(name, version):
    path = models_folder / name / str(version)

    if os.path.exists(path):
        f = [p for p in os.listdir(path) if p.endswith(".keras")]
        if len(f) == 1:
            return tf.keras.models.load_model(path / f[0])


def load_best_model(model_name):
    versions = os.listdir(models_folder / model_name)

    rg = re.compile(r"\d+")
    versions = set([int(s) for s in versions if rg.match(s)])
    versions = list([s for s in versions if str(s) == str(int(s))])
    versions.sort(reverse=True)

    for ver in versions:
        try:
            model = load_model(model_name, ver)
            if not (model is None):
                return model
        except Exception as e:
            print(e)

    raise Exception("Model not found or can't load any version of model")


def best_model_predict(model_name, input_data):
    model = load_best_model(model_name)

    pred_data = model.predict(input_data)
    return pred_data


@app.route("/models/<model>/:predict", methods=["POST"])
def main(model):

    try:
        ## model ##

        data = json.loads(request.data)

        if not (model in os.listdir(models_folder)):
            return {"error": f"not found model: {model}"}

        ## version ##

        versions = os.listdir(models_folder / model)

        rg = re.compile(r"\d+")
        versions = set([int(s) for s in versions if rg.match(s)])
        versions = set([s for s in versions if str(s) == str(int(s))])

        if len(versions) != 0:
            version = str(max(versions))
        else:
            return {"error": f"no versions found for model \"{model}\""}

        # model = load_best_model(model)

        if not ('instances' in data):
            raise Exception('No field "instances" found in your data')
        
        data = np.array(data['instances'])

        pred = best_model_predict(
            model_name=model, 
            input_data=data
        )
        
        return {'prdictions': pred.tolist()}
    except Exception as e:
        print(e)
        return {'error': str(e)}


if __name__ == "__main__":
    # app.run(port=8000)
    http_server = WSGIServer(("", 8502), app)
    print("Server is running. Ctrl+C to interrupt.")
    http_server.serve_forever()


Overwriting server.py


In [7]:
!python server.py

2024-05-27 13:35:37.548941: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-27 13:35:37.552482: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-27 13:35:38.911915: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Current server path: /mnt/o/study/ml_6/8.5/custom_server
Server is running. Ctrl+C to interrupt.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
::ffff:127.0.0.1 - - [2024-05-27 13:36:06] "POST /models/my_model/:predict HTTP/1.1" 200 323 0.328483
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
::ffff:127.0.0.1 - - [2024-05-27 13:36:07] "POST /models/my_model/:predict HTTP/1.1" 200 319 0.232070
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
::ffff:127.0.0.1 - - [2024-05-27 13:36:07] "POST /models/my_model/:predict HTTP/1.1" 200 318 0.228499
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
::ffff:127.0.0.1 - - [2024-05-27 